In [1]:
import pandas as pd
import plotly.express as px
import statsmodels.api as sm

In [2]:
df = pd.read_excel(r"C:\Users\mikej\Desktop\fantrax\fantrax_season_data.xlsm")
# merge = df.copy()
df.columns

Index(['team', 'players_started', 'pars_num', 'bird_num', 'eag_num', 'bog_num',
       'dbog_num', 'alb_num', 'h1_num', 'other_num', 'plc_total', 'week',
       'win_loss', 'opponent', 'fin_1', 'fin_2', 'fin_3', 'fin_4', 'fin_5',
       'fin_6', 'median', 'cuts_made', 'median_delta', 'fin_1_pts',
       'fin_2_pts', 'fin_3_pts', 'fin_4_pts', 'fin_5_pts', 'fin_6_pts',
       'pars_pts', 'bird_pts', 'eag_pts', 'bog_pts', 'dbog_pts', 'alb_pts',
       'h1_pts', 'other_pts', 'plc_pts', 'total_pts', 'total_holes',
       'pp_hole'],
      dtype='object')

In [3]:
team_color={
            "Philly919": 'rgb(14,195,210)',
            "unit_circle": 'rgb(194,139,221)',
            "AlphaWired": 'rgb(247,160,93)',
            "Snead's Foot": 'rgb(70,214,113)',
            "New Team 4": 'rgb(247,94,56)',
            "Team Gamble": 'rgb(38,147,190)',
            "txmoonshine": 'rgb(219,197,48)',
            "Putt Pirates": 'rgb(115,112,106)'
            }

In [4]:
# ###  PER TOURNAMENT AVERAGES  ###

team_stat_avgs = df.groupby('team')[['total_pts','cuts_made','total_holes','pp_hole','bird_num','eag_num','bog_num','dbog_num','plc_pts']].median()#.reset_index()
team_stat_avgs.columns = 'Total Pts','Cuts Made','Holes Played','Pts/Hole','Birdies','Eagles','Bogeys','Doubles','Place Pts'
team_stat_avgs[['Total Pts','Holes Played','Bogeys','Birdies','Place Pts']] = team_stat_avgs[['Total Pts','Holes Played','Bogeys','Birdies','Place Pts']].astype('int')
team_stat_avgs = team_stat_avgs.sort_values('Total Pts',ascending=False).round({'Cuts Made':1,'Pts/Hole':2,'Eagles':1,'Doubles':1})

team_stat_avgs

,Total Pts,Cuts Made,Holes Played,Pts/Hole,Birdies,Eagles,Bogeys,Doubles,Place Pts
team,,,,,,,,,
unit_circle,430,5.5,387,1.04,89,3.0,47,3.5,17
Putt Pirates,413,5.0,396,1.08,86,2.5,45,4.5,34
Team Gamble,366,4.0,360,1.04,81,2.5,46,6.0,25
AlphaWired,364,4.0,342,0.97,77,1.5,42,4.0,22
New Team 4,354,3.5,324,1.05,74,3.0,44,4.0,21
Snead's Foot,344,4.0,360,0.97,75,2.0,45,5.5,12
txmoonshine,338,4.0,342,1.00,75,2.0,48,4.0,16
Philly919,324,4.0,342,0.94,70,1.0,45,3.5,11


In [6]:
### SEASON TO DATE SCORE VS WEEKLY MEDIAN  ###

team_median_deltas = pd.DataFrame(df.groupby('team',as_index=False)['median_delta'].sum()).sort_values(by='median_delta',ascending=False).reset_index(drop=True)

median_delta_bar = px.bar(team_median_deltas,
                          text_auto='.3s',
                          color='team',
                          color_discrete_map=team_color,
                          title='Total Pts vs. Weekly Median<br>thru 11 Weeks',
                          template='plotly_white',
                          labels={'index':'', 'value':''},
                          height=350
                         )

median_delta_bar.update_layout(title_x=.5,legend=dict(title=None))

median_delta_bar.update_xaxes(showticklabels=False)
median_delta_bar.update_yaxes(showticklabels=False, showgrid=False)
median_delta_bar.update_layout(hoverlabel=dict(font_size=18,font_family="Rockwell"),title_x=.45)

median_delta_bar.show()

In [7]:
### WEEKLY SCORE VS WEEKLY MEDIAN FOR EACH TEAM ###

team_weekly_deltas = pd.DataFrame(df[['team','week','median','median_delta']].groupby(['team','week'],as_index=False)[['median_delta','median']].sum())
px.bar(team_weekly_deltas.sort_values('median_delta',ascending=False),
       x='week',
       y='median_delta',
       color='team',
       color_discrete_map=team_color,        
       facet_col='team',
       facet_col_wrap=2,
       facet_col_spacing=.1,
       facet_row_spacing=.16,
       height=800,
       width=800,
       labels={'median_delta':'','week':''},
       template='plotly_white',
       text_auto='.3s'
       ).update_yaxes(showticklabels=False,showgrid=False,gridcolor="#B1A999",tickfont=dict(color='#5A5856', size=10)#,matches=None
       ).update_xaxes(tickfont=dict(color='#5A5856', size=10),title_font=dict(color='#5A5856',size=10),showticklabels=True,tickangle= -90,tickmode='array',tickvals = [1,2,3,4,5,6,7,8,9,10,11,12],ticktext = ['Sony','Amex','Farmers','AT&T','Waste Mgmt','Genesis','Mexico Open','Cognizant','Arnold Palmer','PLAYERS','Valspar','Houston Open'],ticklabelposition='outside'
       ).update_layout(hoverlabel=dict(font_size=14,font_family="Rockwell"),showlegend=False
       ).for_each_annotation(lambda a: a.update(text=a.text.replace("team=", "")))

In [8]:
# wins with negative median delta

temp = df[['team','week','opponent','win_loss','median','total_pts','median_delta']]
temp = temp[(temp.win_loss==True) & (temp.median_delta < 0)]
temp

,team,week,opponent,win_loss,median,total_pts,median_delta
3,txmoonshine,1,Sneads Foot,1,409.3,406.5,-2.8
19,txmoonshine,3,Team Gamble,1,343.8,334.0,-9.8
47,AlphaWired,6,Philly919,1,348.5,342.0,-6.5
49,AlphaWired,7,New Team 4,1,366.0,306.5,-59.5
60,New Team 4,8,Philly919,1,356.0,354.5,-1.5
61,Putt Pirates,8,AlphaWired,1,356.0,344.0,-12.0
69,Philly919,9,txmoonshine,1,378.5,335.5,-43.0
78,txmoonshine,10,Team Gamble,1,350.5,297.0,-53.5
94,Philly919,12,unit_circle,1,334.0,250.5,-83.5


In [17]:
temp_df.columns

Index(['team', 'players_started', 'pars_num', 'bird_num', 'eag_num', 'bog_num',
       'dbog_num', 'alb_num', 'h1_num', 'other_num', 'plc_total', 'week',
       'win_loss', 'opponent', 'fin_1', 'fin_2', 'fin_3', 'fin_4', 'fin_5',
       'fin_6', 'median', 'cuts_made', 'median_delta', 'fin_1_pts',
       'fin_2_pts', 'fin_3_pts', 'fin_4_pts', 'fin_5_pts', 'fin_6_pts',
       'pars_pts', 'bird_pts', 'eag_pts', 'bog_pts', 'dbog_pts', 'alb_pts',
       'h1_pts', 'other_pts', 'plc_pts', 'total_pts', 'total_holes',
       'pp_hole'],
      dtype='object')

In [15]:
### WEEKLY BUBBLES WIN / LOSS  ###

newnames={'False':'Loss','True':'Win'}
temp_df = df.copy()
temp_df['win_loss'] = temp_df['win_loss'].astype('bool')
scatter_fig = px.scatter(temp_df,
                        x='week',
                        y='total_pts',
                        color='win_loss',
                        template='plotly_dark',
                        size='cuts_made',
                        size_max=14,
                        hover_name='team',
                        color_discrete_sequence=px.colors.qualitative.Pastel1,
                        title='Weekly Wins & Losses by Pts Scored',
                        labels={'week':'','total_pts':'Points Scored'},
                        height=800,
                        width=600,                        
                        ).update_layout(hoverlabel=dict(font_size=18,font_family="Rockwell"),showlegend=True,title_x=.35,
                                        legend=dict(orientation='h',yanchor="bottom",y=1,xanchor="center",x=.5,title='',font_color='#5A5856')
                        ).update_xaxes(tickangle= -45,tickvals = [1,2,3,4,5,6,7,8,9,10,11],
                                       ticktext = ['Sony','Amex','Farmers','AT&T','Waste Mgmt','Genesis','Mexico Open','Cognizant','Arnold Palmer','PLAYERS','Valspar'],
                                       tickfont=dict(color='#5A5856', size=13),title_font=dict(color='#5A5856',size=14)
                        ).update_yaxes(tickfont=dict(color='#5A5856', size=13),title_font=dict(color='#5A5856',size=14),tickcolor='darkgrey', gridcolor='darkgrey'
                        ).update_traces(marker=dict(line_color='black')
                        ).for_each_trace(lambda t: t.update(name = newnames[t.name],legendgroup = newnames[t.name],hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])))


scatter_fig.show()#,use_container_width=True, config=config)

In [8]:
### CUTS MADE DISTRIBUTION  ###

newnames={'0':'Loss','1':'Win'}

['#B6E880', '#FF97FF']

cuts_made_hist = px.histogram(df.sort_values('cuts_made',ascending=False),
                    x='cuts_made',
                    text_auto='.2s',
                    title='Win Percentage by Cuts Made',
                    template='plotly_dark',
                    labels={'cuts_made':'Players Thru Cut','count':''},
                    histfunc='count',
                    barnorm='percent',
                    barmode='stack',
                    color='win_loss',
#                     color_discrete_sequence=['red', 'green'],
                    color_discrete_sequence=px.colors.qualitative.Safe,
                    height=350
                             ).update_layout(title_x=.23,legend=dict(title="",x=.45,y=1.2,orientation='h'))

cuts_made_hist.update_layout(title_x=.5, bargap=0.2)
cuts_made_hist.for_each_trace(lambda t: t.update(name = newnames[t.name],legendgroup = newnames[t.name],hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])))

cuts_made_hist.update_xaxes(tickvals = [1,2,3,4,5,6],
                            ticktext = ['1/6','2/6','3/6','4/6','5/6','6/6'])

cuts_made_hist.update_yaxes(showticklabels=False, showgrid=False,visible= False)

In [9]:
### CUTS MADE DISTRIBUTION  ###
df['rounded_percentage'] = (df['cuts_made'] * 100).round().astype(int).astype(str) + '%'

cuts_made_hist = px.histogram(df.sort_values('cuts_made', ascending=False),
                              x='cuts_made',
                              template='plotly_dark',
                              labels={'cuts_made':'', 'count':''},
                              title="%'s Cuts Made",
                              histnorm='percent',
                              color_discrete_sequence=['grey'],
                              height=350,
                              text_auto='.2s'
                             )


cuts_made_hist.update_layout(title_x=.5, bargap=0.2, legend=dict(title="", x=.45, y=1.2, orientation='h'))
cuts_made_hist.update_xaxes(tickvals=[1, 2, 3, 4, 5, 6], ticktext=['1/6', '2/6', '3/6', '4/6', '5/6', '6/6'])
cuts_made_hist.update_yaxes(showticklabels=False, showgrid=False, tickfont=dict(color='#5A5856'),
                             title_font_color='#5A5856', visible=False)

cuts_made_hist.show()


In [31]:
### FINISHING POSITION COMPARISON

finish_medians = round(df[['team','fin_1','fin_2','fin_3','fin_4','fin_5','fin_6']].groupby('team').median(),1).reset_index()
finish_medians.columns = 'Team','Top Finisher','2nd','3rd','4th','5th','Worst Finisher'
melted_finish_medians = finish_medians.melt(id_vars='Team',value_vars=['Top Finisher','2nd','3rd','4th','5th','Worst Finisher'])

# print(finish_medians)

fin_place_scatter = px.scatter(melted_finish_medians,
          x='variable',
          y='value',
          color='Team',
          color_discrete_map=team_color,
          template='plotly_white',
          labels={'value':'Median Finish','variable':''},
#           width=550,
          log_y=True,
          ).update_traces(marker_size=12)

# fin_place_scatter.update_layout(title_x=.5,
#                                legend=dict(
#                                    title=None,
#                                    orientation='h',
#                                    x=0,
#                                    y=1.3
#                                )
#                               )
fin_place_scatter.show()

In [33]:
df

,team,players_started,pars_num,bird_num,eag_num,bog_num,dbog_num,alb_num,h1_num,other_num,...,dbog_pts,alb_pts,h1_pts,other_pts,plc_pts,total_pts,total_holes,pp_hole,rounded_percentage,bb_ratio
0,Snead's Foot,6,189,58,2,32,7,0,0,0,...,-7,0,0,0,2,263.5,288,0.914931,200%,1.812500
1,unit_circle,6,278,98,3,49,4,0,0,0,...,-4,0,0,0,8,436.5,432,1.010417,600%,2.000000
2,New Team 4,3,137,49,4,24,2,0,0,0,...,-2,0,0,0,10,243.5,216,1.127315,300%,2.041667
3,txmoonshine,6,238,81,3,37,1,0,0,0,...,-1,0,0,0,40,406.5,360,1.129167,400%,2.189189
4,Putt Pirates,6,224,81,7,44,4,0,0,0,...,-4,0,0,0,27,412.0,360,1.144444,400%,1.840909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,txmoonshine,6,199,67,0,55,3,0,0,0,...,-3,0,0,0,17,287.0,324,0.885802,300%,1.218182
84,AlphaWired,6,174,63,1,42,7,0,0,1,...,-7,0,0,-1,19,274.0,288,0.951389,200%,1.500000
85,New Team 4,3,140,44,1,27,4,0,0,0,...,-4,0,0,0,30,222.5,216,1.030093,300%,1.629630
86,Philly919,6,165,35,0,44,7,0,0,1,...,-7,0,0,-1,0,157.5,252,0.625000,100%,0.795455


In [28]:
### CORRELATION TO WINS BY STAT - SCATTER PLOTS WITH SLIDER / TOGGLE / RADIO BUTTON FOR EACH STAT

df['bb_ratio'] = df.bird_num / df.bog_num
stats_to_compare = ['bird_num','bb_ratio','median_delta','total_pts','plc_pts','cuts_made','pp_hole','pars_num','eag_num','dbog_num','bog_num']

for stat in stats_to_compare:
    scatter_df = df.groupby(['team'],as_index=False)[[stat,'win_loss']].sum()
    fig = px.scatter(scatter_df,
              x=stat,
              y='win_loss',
              color='team',
              color_discrete_map=team_color,
              template='plotly_white',
              height=400,
              width=600,
              labels={'win_loss':'Wins'},
              title=f"Corr between {stat} vs Wins",
              trendline='ols',trendline_scope='overall',trendline_color_override='black',
              ).update_traces(marker=dict(size=15,line_color='black')
              ).update_layout(showlegend=False)
    results = px.get_trendline_results(fig).px_fit_results.iloc[0].rsquared
    print(f"{stat}")
    print(f"R-Squared Value: {results:.2f}")
    print(fig.show())

bird_num
R-Squared Value: 0.90


None
bb_ratio
R-Squared Value: 0.89


None
median_delta
R-Squared Value: 0.83


None
total_pts
R-Squared Value: 0.83


None
plc_pts
R-Squared Value: 0.68


...

None
pars_num
R-Squared Value: 0.51


None
eag_num
R-Squared Value: 0.18


None
dbog_num
R-Squared Value: 0.03


None
bog_num
R-Squared Value: 0.07


None


In [19]:
px.get_trendline_results(fig).px_fit_results.iloc[0].rsquared

0.8874977727509624

In [21]:
scatter_df = df.groupby(['team'],as_index=False)[[stat,'win_loss']].mean()
fig = px.scatter(scatter_df,
          x=stat,
          y='win_loss',
          color='team',
          color_discrete_map=team_color,
          trendline='ols',trendline_scope='overall',trendline_color_override='black',
      ).update_traces(marker_size=12
      ).update_layout(legend=dict(title=None,orientation='h',x=0,y=1.3))

results = round(px.get_trendline_results(fig).px_fit_results.iloc[0].rsquared,2)
print(results)
fig.show()

0.89


In [ ]:
###  CUTS PER WEEK PAR  ###

cuts_per_week = pd.DataFrame(df.groupby('team')[['cuts_made','total_pts']].sum())
cuts_per_week['cuts_per_week'] = cuts_per_week['cuts_made'] / 10
px.bar(cuts_per_week.sort_values(by='cuts_per_week',ascending=False).round(1),
       y='cuts_per_week',
      log_y=True,
      text_auto=True,
      template='plotly_white',
      color='total_pts',
      height=350,
      color_continuous_scale=px.colors.sequential.Blues)

In [ ]:
### PLAYGROUND BELOW HERE ###

In [ ]:
temp = df.groupby('team')['pars_pts','bird_pts','eag_pts','total_pts'].sum()
temp['bird_eagle_pts'] = temp.bird_pts + temp.eag_pts
temp['portion'] = round(temp.bird_eagle_pts / temp.total_pts,2)
temp['par_portion'] = round(temp.pars_pts / temp.total_pts,2)
temp

In [ ]:
df.groupby(['team']).plc_pts.sum()

In [ ]:
# percentage of time players make cut
df['cuts_per_start'] = df.cuts_made / df.players_started
(
    df.groupby('team')
    ['cuts_per_start']
    .mean()
    .round(2)
)